In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions\\explore'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ssd\\Desktop\\PRO\\End-to-End-MLOps-Developer-Salary-Predictions'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/izouazou/End-to-End-MLOps-Developer-Salary-Predictions.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="izouazou"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1e86993d474c453b78ee31877b61a93590ca1c20"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    preprocessor_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.utils import *

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.CatBoostRegressor
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            preprocessor_path = config.preprocessor_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/izouazou/End-to-End-MLOps-Developer-Salary-Predictions.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)

        model = joblib.load(self.config.model_path)
        preprocessor = joblib.load(self.config.preprocessor_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        test_x = preprocessor.transform(test_x)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            if tracking_url_type_store != "file":

                mlflow.sklearn.log_model(model, "model", registered_model_name="CatBoostModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2023-12-23 23:38:26,567: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-12-23 23:38:26,571: INFO: utils: yaml file: config\params.yaml loaded successfully]
[2023-12-23 23:38:26,576: INFO: utils: yaml file: config\schema.yaml loaded successfully]
[2023-12-23 23:38:26,579: INFO: utils: created directory at: artifacts]
[2023-12-23 23:38:26,581: INFO: utils: created directory at: artifacts/model_evaluation]


[2023-12-23 23:38:28,745: INFO: utils: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\ssd\Desktop\PRO\End-to-End-MLOps-Developer-Salary-Predictions\env\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'CatBoostModel' already exists. Creating a new version of this model...
2023/12/23 23:39:00 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CatBoostModel, version 8
Created version '8' of model 'CatBoostModel'.
